In [1]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import sqlalchemy as db

In [2]:
load_dotenv('.env')
key = os.getenv('key')

In [3]:
top250_url = 'https://www.imdb.com/chart/top/'
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                         "AppleWebKit/605.1.15 (KHTML, like Gecko) "
                         "Version/15.4 Safari/605.1.15"}
response = requests.get(top250_url, headers=headers)
response

<Response [200]>

In [4]:
imdb_html = response.content
soup = BeautifulSoup(imdb_html)
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);


    var ue_err_chan = 'jserr';
(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;c&&c!==k&&c!==m&&c!==n&&c!==p||a.ec++;c&&c!=k||a.ec

In [6]:
imdb_id = []

for i in range(1,251):
    selector = f'#main > div > span > div > div > div.lister > table > tbody > tr:nth-child({i}) > td.titleColumn > a'
    movie_id = soup.select(selector)[0]['href'][7:16]
    imdb_id.append(movie_id)
    
imdb_id

['tt0111161',
 'tt0068646',
 'tt0468569',
 'tt0071562',
 'tt0050083',
 'tt0108052',
 'tt0167260',
 'tt0110912',
 'tt0120737',
 'tt0060196',
 'tt0109830',
 'tt0137523',
 'tt0167261',
 'tt1375666',
 'tt0080684',
 'tt0133093',
 'tt0099685',
 'tt0073486',
 'tt0114369',
 'tt0038650',
 'tt0047478',
 'tt0102926',
 'tt0120815',
 'tt0317248',
 'tt0816692',
 'tt0118799',
 'tt0120689',
 'tt0076759',
 'tt0103064',
 'tt0088763',
 'tt0245429',
 'tt0253474',
 'tt0054215',
 'tt6751668',
 'tt0110413',
 'tt0110357',
 'tt0172495',
 'tt0120586',
 'tt0407887',
 'tt2582802',
 'tt0482571',
 'tt0114814',
 'tt0034583',
 'tt0095327',
 'tt0056058',
 'tt1675434',
 'tt0027977',
 'tt0064116',
 'tt0095765',
 'tt0047396',
 'tt0078748',
 'tt0021749',
 'tt0078788',
 'tt0209144',
 'tt1853728',
 'tt0082971',
 'tt0910970',
 'tt0405094',
 'tt0043014',
 'tt0050825',
 'tt0081505',
 'tt0032553',
 'tt4154756',
 'tt0051201',
 'tt0090605',
 'tt4633694',
 'tt0169547',
 'tt0057012',
 'tt1345836',
 'tt0364569',
 'tt0361748',
 'tt00

In [7]:
imdb_list = []

for movie_id in imdb_id:
    
    movie_url = f'http://www.omdbapi.com/?i={movie_id}&apikey={key}'
    response = requests.get(movie_url)
    
    title = response.json()['Title']
    year = response.json()['Year']
    rated = response.json()['Rated']
    runtime = response.json()['Runtime']
    genres = response.json()['Genre']
    date = response.json()['Released']
    directors = response.json()['Director']
    writers = response.json()['Writer']
    actors = response.json()['Actors']
    languages = response.json()['Language']
    country = response.json()['Country']
    rating = response.json()['imdbRating']
    number_of_ratings = response.json()['imdbVotes']
    movie_df = pd.DataFrame([title, year, runtime, rated, genres, date, directors, writers, actors, languages, country, rating, number_of_ratings], index = ['Title', 'Year', 'Runtime', 'Rated', 'Genres', 'Date', 'Directors', 'Writers', 'Actors', 'Languages', 'Country', 'IMDb Rating', 'Number of ratings']).T
    imdb_list.append(movie_df)

In [8]:
imdb_250 = pd.concat(imdb_list, ignore_index = True)
imdb_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings
0,The Shawshank Redemption,1994,142 min,R,Drama,14 Oct 1994,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",English,United States,9.3,"2,731,095"
1,The Godfather,1972,175 min,R,"Crime, Drama",24 Mar 1972,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan","English, Italian, Latin",United States,9.2,"1,891,559"
2,The Dark Knight,2008,152 min,PG-13,"Action, Crime, Drama",18 Jul 2008,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart","English, Mandarin","United States, United Kingdom",9.0,"2,695,333"
3,The Godfather Part II,1974,202 min,R,"Crime, Drama",18 Dec 1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall","English, Italian, Spanish, Latin, Sicilian",United States,9.0,"1,290,581"
4,12 Angry Men,1957,96 min,Approved,"Crime, Drama",10 Apr 1957,Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",English,United States,9.0,"807,762"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Iron Giant,1999,86 min,PG,"Animation, Action, Adventure",06 Aug 1999,Brad Bird,"Tim McCanlies, Brad Bird, Ted Hughes","Eli Marienthal, Harry Connick Jr., Jennifer An...",English,"United States, Denmark",8.1,"207,273"
246,The Help,2011,146 min,PG-13,Drama,10 Aug 2011,Tate Taylor,"Tate Taylor, Kathryn Stockett","Viola Davis, Emma Stone, Octavia Spencer",English,"United States, India",8.1,"468,668"
247,Aladdin,1992,90 min,G,"Animation, Adventure, Comedy",25 Nov 1992,"Ron Clements, John Musker","Ron Clements, John Musker, Ted Elliott","Scott Weinger, Robin Williams, Linda Larkin",English,"United States, Japan",8.0,"431,684"
248,Dersu Uzala,1975,142 min,G,"Adventure, Biography, Drama",20 Dec 1977,Akira Kurosawa,"Akira Kurosawa, Yuriy Nagibin, Vladimir Arsenev","Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov","Russian, Chinese","Soviet Union, Japan",8.2,"31,466"


In [9]:
ranking = [i for i in range(1, 251)]
ranking

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [10]:
imdb_250['Ranking'] = ranking
imdb_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings,Ranking
0,The Shawshank Redemption,1994,142 min,R,Drama,14 Oct 1994,Frank Darabont,"Stephen King, Frank Darabont","Tim Robbins, Morgan Freeman, Bob Gunton",English,United States,9.3,"2,731,095",1
1,The Godfather,1972,175 min,R,"Crime, Drama",24 Mar 1972,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Marlon Brando, Al Pacino, James Caan","English, Italian, Latin",United States,9.2,"1,891,559",2
2,The Dark Knight,2008,152 min,PG-13,"Action, Crime, Drama",18 Jul 2008,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer","Christian Bale, Heath Ledger, Aaron Eckhart","English, Mandarin","United States, United Kingdom",9.0,"2,695,333",3
3,The Godfather Part II,1974,202 min,R,"Crime, Drama",18 Dec 1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall","English, Italian, Spanish, Latin, Sicilian",United States,9.0,"1,290,581",4
4,12 Angry Men,1957,96 min,Approved,"Crime, Drama",10 Apr 1957,Sidney Lumet,Reginald Rose,"Henry Fonda, Lee J. Cobb, Martin Balsam",English,United States,9.0,"807,762",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Iron Giant,1999,86 min,PG,"Animation, Action, Adventure",06 Aug 1999,Brad Bird,"Tim McCanlies, Brad Bird, Ted Hughes","Eli Marienthal, Harry Connick Jr., Jennifer An...",English,"United States, Denmark",8.1,"207,273",246
246,The Help,2011,146 min,PG-13,Drama,10 Aug 2011,Tate Taylor,"Tate Taylor, Kathryn Stockett","Viola Davis, Emma Stone, Octavia Spencer",English,"United States, India",8.1,"468,668",247
247,Aladdin,1992,90 min,G,"Animation, Adventure, Comedy",25 Nov 1992,"Ron Clements, John Musker","Ron Clements, John Musker, Ted Elliott","Scott Weinger, Robin Williams, Linda Larkin",English,"United States, Japan",8.0,"431,684",248
248,Dersu Uzala,1975,142 min,G,"Adventure, Biography, Drama",20 Dec 1977,Akira Kurosawa,"Akira Kurosawa, Yuriy Nagibin, Vladimir Arsenev","Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov","Russian, Chinese","Soviet Union, Japan",8.2,"31,466",249


In [11]:
boxd_url = 'https://letterboxd.com/dave/list/official-top-250-narrative-feature-films/detail/'
response = requests.get(boxd_url)
response

<Response [200]>

In [12]:
boxd_html = response.content
soup = BeautifulSoup(boxd_html)
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html lang="en" class="ie6 lte9 lte8 lte7 lte6 no-js"> <![endif]-->
<!--[if IE 7 ]>    <html lang="en" class="ie7 lte9 lte8 lte7 no-js"> <![endif]-->
<!--[if IE 8 ]>    <html lang="en" class="ie8 lte9 lte8 no-js"> <![endif]-->
<!--[if IE 9 ]>    <html lang="en" class="ie9 lte9 no-js"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-mobile no-js" id="html" lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=1024" name="viewport"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="A list of 250 films compiled on Letterboxd, including Come and See (1985), Parasite (2019), A Dog's Will (2000), Harakiri (1962) and The Godfather: Part II (1974). About this list: Letterboxd's Top 250 movies, based on the average weighted rating of all Letterboxd users. I removed all stand-up specials, stage plays, concert films, documentaries, shorts, 'collection listings' and other 'rarities', so

In [13]:
boxd_titles = []

for page in range(1,4):
    boxd_url = f'https://letterboxd.com/dave/list/official-top-250-narrative-feature-films/detail/page/{page}/'
    response = requests.get(boxd_url)
    boxd_html = response.content
    soup = BeautifulSoup(boxd_html)
    for title in soup.select('h2 > a'):
        boxd_titles.append(title.text)

boxd_titles

['Come and See',
 'Parasite',
 "A Dog's Will",
 'Harakiri',
 'The Godfather: Part II',
 'The Godfather',
 "The Human Condition III: A Soldier's Prayer",
 '12 Angry Men',
 'Seven Samurai',
 'The Shawshank Redemption',
 'Spirited Away',
 'Central Station',
 'High and Low',
 'City of God',
 'The Dark Knight',
 'Spider-Man: Into the Spider-Verse',
 'Yi Yi',
 'GoodFellas',
 'A Brighter Summer Day',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Neon Genesis Evangelion: The End of Evangelion',
 'The Human Condition I: No Greater Love',
 'Portrait of a Lady on Fire',
 'Ikiru',
 'Everything Everywhere All at Once',
 'Whiplash',
 'The Good, the Bad and the Ugly',
 'La Haine',
 'The Empire Strikes Back',
 'Perfect Blue',
 'There Will Be Blood',
 'Grave of the Fireflies',
 'Stalker',
 'Cinema Paradiso',
 'Ran',
 'The Handmaiden',
 'In the Mood for Love',
 'Persona',
 'Sunset Boulevard',
 'Apocalypse Now',
 'Andrei Rublev',
 'The Passion of Joan of Arc',
 'Woman in the Du

In [14]:
boxd_search = []

for title in boxd_titles:
    boxd_search.append(title.lower().replace(' ', '+'))
    
boxd_search

['come+and+see',
 'parasite',
 "a+dog's+will",
 'harakiri',
 'the+godfather:+part+ii',
 'the+godfather',
 "the+human+condition+iii:+a+soldier's+prayer",
 '12+angry+men',
 'seven+samurai',
 'the+shawshank+redemption',
 'spirited+away',
 'central+station',
 'high+and+low',
 'city+of+god',
 'the+dark+knight',
 'spider-man:+into+the+spider-verse',
 'yi+yi',
 'goodfellas',
 'a+brighter+summer+day',
 "schindler's+list",
 'the+lord+of+the+rings:+the+return+of+the+king',
 'neon+genesis+evangelion:+the+end+of+evangelion',
 'the+human+condition+i:+no+greater+love',
 'portrait+of+a+lady+on+fire',
 'ikiru',
 'everything+everywhere+all+at+once',
 'whiplash',
 'the+good,+the+bad+and+the+ugly',
 'la+haine',
 'the+empire+strikes+back',
 'perfect+blue',
 'there+will+be+blood',
 'grave+of+the+fireflies',
 'stalker',
 'cinema+paradiso',
 'ran',
 'the+handmaiden',
 'in+the+mood+for+love',
 'persona',
 'sunset+boulevard',
 'apocalypse+now',
 'andrei+rublev',
 'the+passion+of+joan+of+arc',
 'woman+in+the+du

In [15]:
boxd_search.index('apur+sansar')
boxd_search[58] = 'the+world+of+apu'

In [16]:
boxd_search.index('i+am+cuba')
boxd_search[73] = 'soy+cuba'

In [17]:
boxd_id = []

for movie in boxd_search:
    if movie == '8½':
        movie_id = 'tt0056801'
        boxd_id.append(movie_id)
    elif movie == 'three+colours:+red':
        movie_id = 'tt0111495'
        boxd_id.append(movie_id)
    elif movie == 'm':
        movie_id = 'tt0022100'
        boxd_id.append(movie_id)
    elif movie == 'evangelion:+3.0+1.0+thrice+upon+a+time':
        movie_id = 'tt2458948'
        boxd_id.append(movie_id)
    elif movie == 'z':
        movie_id = 'tt0065234'
        boxd_id.append(movie_id)
    elif movie == 'gangs+of+wasseypur+-+part+2':
        movie_id = 'tt1954470'
        boxd_id.append(movie_id)
    else:
        movie_url = f'https://www.omdbapi.com/?s={movie}&apikey={key}'
        response = requests.get(movie_url)
        movie_id = response.json()['Search'][0]['imdbID']
        boxd_id.append(movie_id)

boxd_id

['tt0091251',
 'tt6751668',
 'tt0271383',
 'tt0056058',
 'tt0071562',
 'tt0068646',
 'tt0055233',
 'tt0050083',
 'tt0047478',
 'tt0111161',
 'tt0245429',
 'tt0140888',
 'tt0057565',
 'tt0317248',
 'tt0468569',
 'tt4633694',
 'tt0244316',
 'tt0099685',
 'tt0101985',
 'tt0108052',
 'tt0167260',
 'tt0169858',
 'tt0053114',
 'tt8613070',
 'tt0044741',
 'tt6710474',
 'tt2582802',
 'tt0060196',
 'tt0113247',
 'tt0080684',
 'tt0156887',
 'tt0469494',
 'tt0095327',
 'tt0079944',
 'tt0095765',
 'tt0089881',
 'tt4016934',
 'tt0118694',
 'tt0060827',
 'tt0158253',
 'tt0078788',
 'tt0060107',
 'tt0019254',
 'tt0058625',
 'tt0053115',
 'tt0077711',
 'tt0046438',
 'tt0054042',
 'tt0087884',
 'tt0119698',
 'tt0097216',
 'tt0167261',
 'tt0050825',
 'tt0347149',
 'tt0120737',
 'tt0112471',
 'tt0089603',
 'tt0111341',
 'tt0052572',
 'tt0084787',
 'tt0053604',
 'tt0364569',
 'tt0054407',
 'tt0156794',
 'tt0047396',
 'tt0056322',
 'tt0050634',
 'tt0102926',
 'tt0083922',
 'tt0353969',
 'tt0381681',
 'tt23

In [18]:
boxd_list = []

for movie_id in boxd_id:

    movie_url = f'http://www.omdbapi.com/?i={movie_id}&apikey={key}'
    response = requests.get(movie_url)
    
    title = response.json()['Title']
    year = response.json()['Year']
    rated = response.json()['Rated']
    runtime = response.json()['Runtime']
    genres = response.json()['Genre']
    date = response.json()['Released']
    directors = response.json()['Director']
    writers = response.json()['Writer']
    actors = response.json()['Actors']
    languages = response.json()['Language']
    country = response.json()['Country']
    rating = response.json()['imdbRating']
    number_of_ratings = response.json()['imdbVotes']
    movie_df = pd.DataFrame([title, year, runtime, rated, genres, date, directors, writers, actors, languages, country, rating, number_of_ratings], index = ['Title', 'Year', 'Runtime', 'Rated', 'Genres', 'Date', 'Directors', 'Writers', 'Actors', 'Languages', 'Country', 'IMDb Rating', 'Number of ratings']).T
    boxd_list.append(movie_df)

In [19]:
boxd_250 = pd.concat(boxd_list, ignore_index = True)
boxd_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings
0,Come and See,1985,142 min,Not Rated,"Drama, Thriller, War",03 Sep 1985,Elem Klimov,"Ales Adamovich, Elem Klimov","Aleksey Kravchenko, Olga Mironova, Liubomiras ...","Belarusian, Russian, German",Soviet Union,8.4,"83,760"
1,Parasite,2019,132 min,R,"Drama, Thriller",08 Nov 2019,Bong Joon Ho,"Bong Joon Ho, Han Jin-won","Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong","Korean, English",South Korea,8.5,"837,082"
2,A Dog's Will,2000,104 min,N/A,"Comedy, Drama, Fantasy",10 Sep 2000,Guel Arraes,"Guel Arraes, Adriana Falcão, João Falcão","Matheus Nachtergaele, Selton Mello, Rogério Ca...","Portuguese, Latin",Brazil,8.6,"15,369"
3,Harakiri,1962,133 min,Not Rated,"Action, Drama, Mystery",04 Aug 1964,Masaki Kobayashi,"Yasuhiko Takiguchi, Shinobu Hashimoto","Tatsuya Nakadai, Akira Ishihama, Shima Iwashita",Japanese,Japan,8.6,"59,979"
4,The Godfather Part II,1974,202 min,R,"Crime, Drama",18 Dec 1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall","English, Italian, Spanish, Latin, Sicilian",United States,9.0,"1,290,581"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Kumbalangi Nights,2019,135 min,Not Rated,"Comedy, Drama, Romance",21 Feb 2019,Madhu C. Narayanan,Syam Pushkaran,"Shane Nigam, Soubin Shahir, Fahadh Faasil",Malayalam,India,8.5,"15,524"
246,Samurai Rebellion,1967,128 min,Not Rated,Drama,01 Dec 1967,Masaki Kobayashi,"Shinobu Hashimoto, Yasuhiko Takiguchi","Toshirô Mifune, Yôko Tsukasa, Gô Katô",Japanese,Japan,8.3,"13,222"
247,Some Like It Hot,1959,121 min,Passed,"Comedy, Music, Romance",19 Mar 1959,Billy Wilder,"Billy Wilder, I.A.L. Diamond, Robert Thoeren","Marilyn Monroe, Tony Curtis, Jack Lemmon",English,United States,8.2,"271,609"
248,The Social Network,2010,120 min,PG-13,"Biography, Drama",01 Oct 2010,David Fincher,"Aaron Sorkin, Ben Mezrich","Jesse Eisenberg, Andrew Garfield, Justin Timbe...","English, French",United States,7.8,"717,677"


In [20]:
boxd_250['Ranking'] = ranking
boxd_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings,Ranking
0,Come and See,1985,142 min,Not Rated,"Drama, Thriller, War",03 Sep 1985,Elem Klimov,"Ales Adamovich, Elem Klimov","Aleksey Kravchenko, Olga Mironova, Liubomiras ...","Belarusian, Russian, German",Soviet Union,8.4,"83,760",1
1,Parasite,2019,132 min,R,"Drama, Thriller",08 Nov 2019,Bong Joon Ho,"Bong Joon Ho, Han Jin-won","Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong","Korean, English",South Korea,8.5,"837,082",2
2,A Dog's Will,2000,104 min,N/A,"Comedy, Drama, Fantasy",10 Sep 2000,Guel Arraes,"Guel Arraes, Adriana Falcão, João Falcão","Matheus Nachtergaele, Selton Mello, Rogério Ca...","Portuguese, Latin",Brazil,8.6,"15,369",3
3,Harakiri,1962,133 min,Not Rated,"Action, Drama, Mystery",04 Aug 1964,Masaki Kobayashi,"Yasuhiko Takiguchi, Shinobu Hashimoto","Tatsuya Nakadai, Akira Ishihama, Shima Iwashita",Japanese,Japan,8.6,"59,979",4
4,The Godfather Part II,1974,202 min,R,"Crime, Drama",18 Dec 1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo","Al Pacino, Robert De Niro, Robert Duvall","English, Italian, Spanish, Latin, Sicilian",United States,9.0,"1,290,581",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Kumbalangi Nights,2019,135 min,Not Rated,"Comedy, Drama, Romance",21 Feb 2019,Madhu C. Narayanan,Syam Pushkaran,"Shane Nigam, Soubin Shahir, Fahadh Faasil",Malayalam,India,8.5,"15,524",246
246,Samurai Rebellion,1967,128 min,Not Rated,Drama,01 Dec 1967,Masaki Kobayashi,"Shinobu Hashimoto, Yasuhiko Takiguchi","Toshirô Mifune, Yôko Tsukasa, Gô Katô",Japanese,Japan,8.3,"13,222",247
247,Some Like It Hot,1959,121 min,Passed,"Comedy, Music, Romance",19 Mar 1959,Billy Wilder,"Billy Wilder, I.A.L. Diamond, Robert Thoeren","Marilyn Monroe, Tony Curtis, Jack Lemmon",English,United States,8.2,"271,609",248
248,The Social Network,2010,120 min,PG-13,"Biography, Drama",01 Oct 2010,David Fincher,"Aaron Sorkin, Ben Mezrich","Jesse Eisenberg, Andrew Garfield, Justin Timbe...","English, French",United States,7.8,"717,677",249


In [21]:
bfi_url = 'https://www.bfi.org.uk/sight-and-sound/greatest-films-all-time'
response = requests.get(bfi_url)
response

<Response [200]>

In [22]:
bfi_html = response.content
soup = BeautifulSoup(bfi_html)
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="
      In 1952, the Sight and Sound team had the novel idea of asking critics to name the greatest films of all time. The tradition became decennial, increasing in size and prestige as the decades passed.
      The Sight and Sound poll is now a major bellwether of critical opinion on cinema and this year’s edition (its eighth) is the largest ever, with 1,639 participating critics, programmers, curators, archivists and academics each submitting their top ten ballot. What has risen up the ranks? What has fallen? Has 2012’s winner Vertigo held on to its title? Find out below." data-react-helmet="true" name="description"/>
  <meta content="index, follow" data-react-helmet="true" name="robots"/>
  <meta content="width=device-width, initial-scale=1" data-react-helmet="true" name="viewport"/>
  <meta content="#783df6" data-react-helmet="true" name="msapplication-TileColor"/>
  <meta content="/dist/server/a67a7d9041d0ee52635dff7f0110378

In [23]:
bfi_titles = []

for title in soup.find_all('h1'):
    bfi_titles.append(title.text.strip())

bfi_titles.remove('The Greatest Films of All Time')
bfi_titles.reverse()
bfi_titles

['Jeanne Dielman  23, Quai du Commerce  1080 Bruxelles',
 'Vertigo',
 'Citizen Kane',
 'Tokyo Story',
 'In the Mood for Love',
 '2001: A Space Odyssey',
 'Beau travail',
 'Mulholland Dr.',
 'Man with a Movie Camera',
 "Singin' in the Rain",
 'Sunrise: A Song of Two Humans',
 'The Godfather',
 'La Règle du jeu',
 'Cléo from 5 to 7',
 'The Searchers',
 'Meshes of the Afternoon',
 'Close-up',
 'Persona',
 'Apocalypse Now',
 'Seven Samurai',
 'The Passion of Joan of Arc',
 'Late Spring',
 'Playtime',
 'Do the Right Thing',
 'Au hasard Balthazar',
 'The Night of the Hunter',
 'Shoah',
 'Daisies',
 'Taxi Driver',
 'Portrait of a Lady on Fire',
 'Mirror',
 'Psycho',
 '8½',
 "L'Atalante",
 'Pather Panchali',
 'City Lights',
 'M',
 'Some Like It Hot',
 'À bout de souffle',
 'Rear Window',
 'Bicycle Thieves',
 'Rashomon',
 'Killer of Sheep',
 'Stalker',
 'The Battle of Algiers',
 'North by Northwest',
 'Barry Lyndon',
 'Wanda',
 'Ordet',
 'The 400 Blows',
 'The Piano',
 'Fear Eats the Soul',
 'N

In [24]:
bfi_rank = []

for ranking in soup.find_all('p', attrs = {'class': "ResultsPage__Rank-sc-of10co-1 kFoTQs PreviewCard__label"}):
    if ranking.text.startswith('='):
        bfi_rank.append(ranking.text.replace('=', ''))
    else:
        bfi_rank.append(ranking.text)
    
bfi_rank.reverse()
bfi_rank

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '21',
 '23',
 '24',
 '25',
 '25',
 '27',
 '28',
 '29',
 '30',
 '31',
 '31',
 '31',
 '34',
 '35',
 '36',
 '36',
 '38',
 '38',
 '38',
 '41',
 '41',
 '43',
 '43',
 '45',
 '45',
 '45',
 '48',
 '48',
 '50',
 '50',
 '52',
 '52',
 '54',
 '54',
 '54',
 '54',
 '54',
 '59',
 '60',
 '60',
 '60',
 '63',
 '63',
 '63',
 '66',
 '67',
 '67',
 '67',
 '67',
 '67',
 '72',
 '72',
 '72',
 '75',
 '75',
 '75',
 '78',
 '78',
 '78',
 '78',
 '78',
 '78',
 '78',
 '85',
 '85',
 '85',
 '88',
 '88',
 '90',
 '90',
 '90',
 '90',
 '90',
 '95',
 '95',
 '95',
 '95',
 '95',
 '95',
 '101',
 '101',
 '101',
 '104',
 '104',
 '104',
 '104',
 '108',
 '108',
 '108',
 '108',
 '108',
 '108',
 '114',
 '114',
 '114',
 '114',
 '118',
 '118',
 '118',
 '118',
 '122',
 '122',
 '122',
 '122',
 '122',
 '122',
 '128',
 '129',
 '129',
 '129',
 '129',
 '133',
 '133',
 '133',
 '136',
 '136',
 '136',
 '136

In [25]:
bfi_search = []

for title in bfi_titles:
    bfi_search.append(title.lower().replace(' ', '+'))
    
bfi_search

['jeanne+dielman++23,+quai+du+commerce++1080+bruxelles',
 'vertigo',
 'citizen+kane',
 'tokyo+story',
 'in+the+mood+for+love',
 '2001:+a+space+odyssey',
 'beau+travail',
 'mulholland+dr.',
 'man+with+a+movie+camera',
 "singin'+in+the+rain",
 'sunrise:+a+song+of+two+humans',
 'the+godfather',
 'la+règle+du+jeu',
 'cléo+from+5+to+7',
 'the+searchers',
 'meshes+of+the+afternoon',
 'close-up',
 'persona',
 'apocalypse+now',
 'seven+samurai',
 'the+passion+of+joan+of+arc',
 'late+spring',
 'playtime',
 'do+the+right+thing',
 'au+hasard+balthazar',
 'the+night+of+the+hunter',
 'shoah',
 'daisies',
 'taxi+driver',
 'portrait+of+a+lady+on+fire',
 'mirror',
 'psycho',
 '8½',
 "l'atalante",
 'pather+panchali',
 'city+lights',
 'm',
 'some+like+it+hot',
 'à+bout+de+souffle',
 'rear+window',
 'bicycle+thieves',
 'rashomon',
 'killer+of+sheep',
 'stalker',
 'the+battle+of+algiers',
 'north+by+northwest',
 'barry+lyndon',
 'wanda',
 'ordet',
 'the+400+blows',
 'the+piano',
 'fear+eats+the+soul',
 'n

In [27]:
bfi_search.index('sunrise:+a+song+of+two+humans')
bfi_search[10] = 'sunrise'

In [28]:
bfi_search.index('touki+bouki')
bfi_search[65] = 'journey+of+the+hyena'

In [29]:
bfi_search.index('my+neighbour+totoro')
bfi_search[71] = 'my+neighbor+totoro'

In [30]:
bfi_search.index('ugetsu+monogatari')
bfi_search[91] = 'ugetsu'

In [31]:
bfi_search.index('sans+toit+ni+loi')
bfi_search[102] = 'vagabond'

In [32]:
bfi_search.index('la+maman+et+la+putain')
bfi_search[103] = 'the+mother+and+the+whore'

In [33]:
bfi_search.index('aguirre,+wrath+of+god')
bfi_search[117] = 'aguirre'

In [34]:
bfi_search.index('les+enfants+du+paradis')
bfi_search[136] = 'children+of+paradise'

In [35]:
bfi_search.index('west+indies:+the+fugitive+slaves+of+liberty')
bfi_search[152] = 'west+indies'

In [36]:
bfi_search.index('meghe+dhaka+tara')
bfi_search[154] = 'the+cloud-capped+star'

In [37]:
bfi_search.index('todo+sobre+mi+madre')
bfi_search[159] = 'all+about+my+mother'

In [38]:
bfi_search.index('les+demoiselles+de+rochefort')
bfi_search[193] = 'the+young+girls+of+rochefort'

In [39]:
bfi_search.index("l'+eclisse")
bfi_search[206] = "l'eclisse"

In [40]:
bfi_search.index("l'armée+des+ombres")
bfi_search[216] = 'army-of-shadows'

In [41]:
bfi_search.index("europa+'51")
bfi_search[236] = "europe+'51"

In [42]:
bfi_search.index('the+quince+tree+sun')
bfi_search[243] = 'dream+of+light'

In [43]:
bfi_id = []

for movie in bfi_search:
    try:
        if movie == 'jeanne+dielman++23,+quai+du+commerce++1080+bruxelles':
            movie_id = 'tt0073198'
            bfi_id.append(movie_id)
        elif movie == '8½':
            movie_id = 'tt0056801'
            bfi_id.append(movie_id)
        elif movie == 'm':
            movie_id = 'tt0022100'
            bfi_id.append(movie_id)
        elif movie == 'twin+peaks:+the+return':
            movie_id = 'tt4093826'
            bfi_id.append(movie_id)
        elif movie == 'vivre+sa+vie++film+en+douze+tableaux':
            movie_id = 'tt0056663'
            bfi_id.append(movie_id)
        elif movie == 'mad+max++fury+road':
            movie_id = 'tt1392190'
            bfi_id.append(movie_id)
        elif movie == 'twin+peaks++fire+walk+with+me':
            movie_id = 'tt0105665'
            bfi_id.append(movie_id)
        else:
            movie_url = f'https://www.omdbapi.com/?s={movie}&apikey={key}'
            response = requests.get(movie_url)
            movie_id = response.json()['Search'][0]['imdbID']
            bfi_id.append(movie_id)
    except:
        print(movie)

bfi_id

['tt0073198',
 'tt0052357',
 'tt0033467',
 'tt0046438',
 'tt0118694',
 'tt0062622',
 'tt0209933',
 'tt1619856',
 'tt0019760',
 'tt0045152',
 'tt0112471',
 'tt0068646',
 'tt2944140',
 'tt0055852',
 'tt0049730',
 'tt0036154',
 'tt0100234',
 'tt0060827',
 'tt0078788',
 'tt0047478',
 'tt0019254',
 'tt0041154',
 'tt0062136',
 'tt0097216',
 'tt0060138',
 'tt0048424',
 'tt0090015',
 'tt0925266',
 'tt0075314',
 'tt8613070',
 'tt2085059',
 'tt0054215',
 'tt0056801',
 'tt0024844',
 'tt0048473',
 'tt0021749',
 'tt0022100',
 'tt0053291',
 'tt0353925',
 'tt0047396',
 'tt0040522',
 'tt0042876',
 'tt0076263',
 'tt0079944',
 'tt0058946',
 'tt0053125',
 'tt0072684',
 'tt0095159',
 'tt0048452',
 'tt0053198',
 'tt0107822',
 'tt0071141',
 'tt0076452',
 'tt0083658',
 'tt0015648',
 'tt5235716',
 'tt0015324',
 'tt0053604',
 'tt0084628',
 'tt4975722',
 'tt0053779',
 'tt0104057',
 'tt0034583',
 'tt0099685',
 'tt0041959',
 'tt0070820',
 'tt0345128',
 'tt0056119',
 'tt0060107',
 'tt0017136',
 'tt0040725',
 'tt00

In [44]:
bfi_list = []

for movie_id in bfi_id:
    movie_url = f'http://www.omdbapi.com/?i={movie_id}&apikey={key}'
    response = requests.get(movie_url)
    
    title = response.json()['Title']
    year = response.json()['Year']
    rated = response.json()['Rated']
    runtime = response.json()['Runtime']
    genres = response.json()['Genre']
    date = response.json()['Released']
    directors = response.json()['Director']
    writers = response.json()['Writer']
    actors = response.json()['Actors']
    languages = response.json()['Language']
    country = response.json()['Country']
    rating = response.json()['imdbRating']
    number_of_ratings = response.json()['imdbVotes']
    movie_df = pd.DataFrame([title, year, runtime, rated, genres, date, directors, writers, actors, languages, country, rating, number_of_ratings], index = ['Title', 'Year', 'Runtime', 'Rated', 'Genres', 'Date', 'Directors', 'Writers', 'Actors', 'Languages', 'Country', 'IMDb Rating', 'Number of ratings']).T
    bfi_list.append(movie_df)

In [45]:
bfi_250 = pd.concat(bfi_list, ignore_index = True)
bfi_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings
0,"Jeanne Dielman, 23, quai du commerce, 1080 Bru...",1975,202 min,Not Rated,Drama,21 Jan 1976,Chantal Akerman,Chantal Akerman,"Delphine Seyrig, Jan Decorte, Henri Storck",French,"Belgium, France",7.6,"11,549"
1,Vertigo,1958,128 min,PG,"Mystery, Romance, Thriller",22 May 1958,Alfred Hitchcock,"Alec Coppel, Samuel A. Taylor, Pierre Boileau","James Stewart, Kim Novak, Barbara Bel Geddes",English,United States,8.3,"407,861"
2,Citizen Kane,1941,119 min,PG,"Drama, Mystery",05 Sep 1941,Orson Welles,"Herman J. Mankiewicz, Orson Welles, John Houseman","Orson Welles, Joseph Cotten, Dorothy Comingore","English, Italian",United States,8.3,"447,879"
3,Tokyo Story,1953,136 min,Not Rated,Drama,13 Mar 1972,Yasujirô Ozu,"Kôgo Noda, Yasujirô Ozu","Chishû Ryû, Chieko Higashiyama, Sô Yamamura","Japanese, English",Japan,8.2,"63,518"
4,In the Mood for Love,2000,98 min,PG,"Drama, Romance",09 Mar 2001,Kar-Wai Wong,Kar-Wai Wong,"Tony Leung Chiu-wai, Maggie Cheung, Siu Ping-Lam","Cantonese, Shanghainese, French, Spanish","Hong Kong, France",8.1,"155,451"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Planet Earth,2006,538 min,TV-PG,Documentary,25 Mar 2007,N/A,N/A,"Sigourney Weaver, David Attenborough, Nikolay ...",English,United Kingdom,9.4,"212,483"
260,Annie Hall,1977,93 min,PG,"Comedy, Romance",20 Apr 1977,Woody Allen,"Woody Allen, Marshall Brickman","Woody Allen, Diane Keaton, Tony Roberts","English, German",United States,8.0,"269,350"
261,Sullivan's Travels,1941,90 min,Passed,"Adventure, Comedy, Drama",06 Feb 1942,Preston Sturges,Preston Sturges,"Joel McCrea, Veronica Lake, Robert Warwick",English,United States,7.9,"27,237"
262,Pandora's Box,1929,109 min,Not Rated,"Crime, Drama, Romance",22 Feb 1930,Georg Wilhelm Pabst,"Frank Wedekind, Ladislaus Vajda, Joseph Fleisler","Louise Brooks, Fritz Kortner, Francis Lederer","None, German",Germany,7.8,"11,241"


In [46]:
bfi_250['Ranking'] = bfi_rank
bfi_250

,Title,Year,Runtime,Rated,Genres,Date,Directors,Writers,Actors,Languages,Country,IMDb Rating,Number of ratings,Ranking
0,"Jeanne Dielman, 23, quai du commerce, 1080 Bru...",1975,202 min,Not Rated,Drama,21 Jan 1976,Chantal Akerman,Chantal Akerman,"Delphine Seyrig, Jan Decorte, Henri Storck",French,"Belgium, France",7.6,"11,549",1
1,Vertigo,1958,128 min,PG,"Mystery, Romance, Thriller",22 May 1958,Alfred Hitchcock,"Alec Coppel, Samuel A. Taylor, Pierre Boileau","James Stewart, Kim Novak, Barbara Bel Geddes",English,United States,8.3,"407,861",2
2,Citizen Kane,1941,119 min,PG,"Drama, Mystery",05 Sep 1941,Orson Welles,"Herman J. Mankiewicz, Orson Welles, John Houseman","Orson Welles, Joseph Cotten, Dorothy Comingore","English, Italian",United States,8.3,"447,879",3
3,Tokyo Story,1953,136 min,Not Rated,Drama,13 Mar 1972,Yasujirô Ozu,"Kôgo Noda, Yasujirô Ozu","Chishû Ryû, Chieko Higashiyama, Sô Yamamura","Japanese, English",Japan,8.2,"63,518",4
4,In the Mood for Love,2000,98 min,PG,"Drama, Romance",09 Mar 2001,Kar-Wai Wong,Kar-Wai Wong,"Tony Leung Chiu-wai, Maggie Cheung, Siu Ping-Lam","Cantonese, Shanghainese, French, Spanish","Hong Kong, France",8.1,"155,451",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Planet Earth,2006,538 min,TV-PG,Documentary,25 Mar 2007,N/A,N/A,"Sigourney Weaver, David Attenborough, Nikolay ...",English,United Kingdom,9.4,"212,483",243
260,Annie Hall,1977,93 min,PG,"Comedy, Romance",20 Apr 1977,Woody Allen,"Woody Allen, Marshall Brickman","Woody Allen, Diane Keaton, Tony Roberts","English, German",United States,8.0,"269,350",243
261,Sullivan's Travels,1941,90 min,Passed,"Adventure, Comedy, Drama",06 Feb 1942,Preston Sturges,Preston Sturges,"Joel McCrea, Veronica Lake, Robert Warwick",English,United States,7.9,"27,237",243
262,Pandora's Box,1929,109 min,Not Rated,"Crime, Drama, Romance",22 Feb 1930,Georg Wilhelm Pabst,"Frank Wedekind, Ladislaus Vajda, Joseph Fleisler","Louise Brooks, Fritz Kortner, Francis Lederer","None, German",Germany,7.8,"11,241",243


In [47]:
user = os.getenv('user')
password = os.getenv('password')
url_banco = '127.0.0.1'
nome_db = 'top_250_films'
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"
engine = db.create_engine(conn_str)

In [48]:
imdb_250.to_sql('imdb_250', engine, index = False, if_exists = 'replace')
boxd_250.to_sql('boxd_250', engine, index = False, if_exists = 'replace')
bfi_250.to_sql('bfi_250', engine, index = False, if_exists = 'replace')

264